# Check causal tracing results

In [1]:
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import os
import math
import scipy
import random

from experiments.causal_trace import plot_trace_heatmap, plot_trace_heatmap_from_values, plot_normalized_trace_heatmap_from_values

In [2]:
query_file = "/cephyr/users/lovhag/Alvis/projects/rome/data/P138_gpt2_xl_preds.jsonl"
results_folder = "/cephyr/users/lovhag/Alvis/projects/rome/data/results/gpt2-xl/P138/causal_trace_pararel_1890840"

In [3]:
queries = pd.DataFrame()
with open(query_file) as f:
    for line in f:
        queries = queries.append(json.loads(line), ignore_index=True)
        
queries.head()

,attribute,candidate_p,candidate_prediction,candidate_rank,gold_p,gold_rank,known_id,prediction,prediction_p,prompt,relation_id,subject,template,top10_tokens,top10_tokens_probs
0,Aristotle,0.003646,Aristotle,29.0,0.003646,29.0,0.0,the,0.274377,aristotelianism is named after,P138,aristotelianism,{} is named after,"[ the, a, its, him, Saint, Pierre, this,...","[0.2743774652481079, 0.06579102575778961, 0.04..."
1,Aristotle,0.004157,Aristotle,21.0,0.004157,21.0,1.0,the,0.273168,aristotelianism was named after,P138,aristotelianism,{} was named after,"[ the, a, its, him, Saint, Pierre, this,...","[0.2731681764125824, 0.06449723988771439, 0.04..."
2,Aristotle,0.004547,Aristotle,23.0,0.004547,23.0,2.0,the,0.280110,aristotelianism is named for,P138,aristotelianism,{} is named for,"[ the, a, its, him, Saint, Pierre, this,...","[0.28011029958724976, 0.06155542656779289, 0.0..."
3,Aristotle,0.005192,Aristotle,17.0,0.005192,17.0,3.0,the,0.286806,aristotelianism was named for,P138,aristotelianism,{} was named for,"[ the, a, its, him, Saint, Pierre, this,...","[0.28680598735809326, 0.05883852764964104, 0.0..."
4,Aristotle,0.004765,French,19.0,0.002834,29.0,4.0,the,0.333873,"aristotelianism, which is named after",P138,aristotelianism,"{}, which is named after","[ the, a, its, him, Saint, Pierre, this,...","[0.33387309312820435, 0.05952830985188484, 0.0..."


In [12]:
len(queries[queries.candidate_rank<10])

3188

In [13]:
len(queries)

6570

In [14]:
queries[queries.candidate_rank<10]

,attribute,candidate_p,candidate_prediction,candidate_rank,gold_p,gold_rank,known_id,prediction,prediction_p,prompt,relation_id,subject,template,top10_tokens,top10_tokens_probs
45,Paul,0.021233,Paul,6.0,0.021233,6.0,45.0,the,0.334310,Pauline epistles is named after,P138,Pauline epistles,{} is named after,"[ the, her, a, St, Saint, Mary, Paul, J...","[0.3343103229999542, 0.15753313899040222, 0.06..."
46,Paul,0.018955,Paul,6.0,0.018955,6.0,46.0,the,0.279396,Pauline epistles was named after,P138,Pauline epistles,{} was named after,"[ the, her, a, St, Saint, Mary, Paul, J...","[0.27939626574516296, 0.2490142583847046, 0.05..."
47,Paul,0.020659,Paul,5.0,0.020659,5.0,47.0,the,0.349007,Pauline epistles is named for,P138,Pauline epistles,{} is named for,"[ the, her, a, St, Saint, Mary, Paul, J...","[0.3490067422389984, 0.13481958210468292, 0.06..."
48,Paul,0.017353,Paul,6.0,0.017353,6.0,48.0,the,0.320781,Pauline epistles was named for,P138,Pauline epistles,{} was named for,"[ the, her, a, St, Saint, Mary, Paul, J...","[0.3207806348800659, 0.20654287934303284, 0.05..."
49,Paul,0.023575,Paul,5.0,0.023575,5.0,49.0,the,0.311909,"Pauline epistles, which is named after",P138,Pauline epistles,"{}, which is named after","[ the, her, a, St, Saint, Mary, Paul, J...","[0.31190869212150574, 0.22290809452533722, 0.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6522,Tokyo,0.010936,Tokyo,6.0,0.010936,6.0,6522.0,the,0.456713,"Tokyo Rose, which is called after",P138,Tokyo Rose,"{}, which is called after","[ the, a, her, an, one, Tokyo, its, Ros...","[0.45671284198760986, 0.19537846744060516, 0.0..."
6523,Tokyo,0.009234,Tokyo,7.0,0.009234,7.0,6523.0,the,0.418663,"Tokyo Rose, which was called after",P138,Tokyo Rose,"{}, which was called after","[ the, a, her, an, one, Tokyo, its, Ros...","[0.4186630845069885, 0.15778322517871857, 0.04..."
6561,biodiversity,0.003748,Earth,6.0,0.002052,12.0,6561.0,the,0.691139,"International Day for Biological Diversity, wh...",P138,International Day for Biological Diversity,"{}, which is named for","[ the, a, an, Charles, Maria, Sir, Jane,...","[0.6911388635635376, 0.0768628865480423, 0.014..."
6562,biodiversity,0.004898,Earth,6.0,0.001536,34.0,6562.0,the,0.558512,"International Day for Biological Diversity, wh...",P138,International Day for Biological Diversity,"{}, which was named for","[ the, a, an, Charles, Maria, Sir, Jane,...","[0.5585119128227234, 0.07489259541034698, 0.01..."


In [7]:
row["top10_tokens"]

[' the',
 ' a',
 ' its',
 ' him',
 ' Saint',
 ' Pierre',
 ' this',
 ' Jean',
 ' one',
 ' an']

In [15]:
def plot_trace_heatmap_from_values(differences, low_score, high_score, answer, kind, window, labels, ax,
                                   savepdf, title, xlabel, modelname):
    with plt.rc_context():
        #fig, ax = plt.subplots(figsize=(3.5, 2), dpi=200)
        h = ax.pcolor(
            differences,
            cmap={None: "Purples", "None": "Purples", "mlp": "Greens", "attn": "Reds"}[
                kind
            ],
            vmin=low_score,
            vmax=high_score
        )
        ax.invert_yaxis()
        ax.set_yticks([0.5 + i for i in range(len(differences))])
        ax.set_xticks([0.5 + i for i in range(0, differences.shape[1] - 6, 5)])
        ax.set_xticklabels(list(range(0, differences.shape[1] - 6, 5)))
        ax.set_yticklabels(labels, fontsize=8)
        cb = plt.colorbar(h, ax=ax)
        cb.ax.tick_params(labelsize=8)
        if title is not None:
            ax.set_title(title)
        if xlabel is not None:
            ax.set_xlabel(xlabel)
        elif answer is not None:
            # The following should be cb.ax.set_xlabel, but this is broken in matplotlib 3.5.1.
            cb.ax.set_title(f"p({str(answer).strip()})", y=-0.16, fontsize=8)
        if savepdf:
            os.makedirs(os.path.dirname(savepdf), exist_ok=True)
            plt.savefig(savepdf, bbox_inches="tight")
            plt.close()
        #else:
        #    plt.show()
            
def plot_trace_heatmap(result, ax, savepdf=None, title=None, xlabel=None, modelname=None):
    differences = result["scores"]
    low_score = result["low_score"]
    high_score = None #result["high_score"]
    answer = result["answer"]
    kind = (
        None
        if (not result["kind"] or result["kind"] == "None")
        else str(result["kind"])
    )
    window = result.get("window", 10)
    labels = list(result["input_tokens"])
    for i in range(*result["subject_range"]):
        labels[i] = labels[i] + "*"

    plot_trace_heatmap_from_values(differences, low_score, high_score, answer, kind, window, labels, ax,
                                   savepdf, title, xlabel, modelname)

In [16]:
def get_results_for_token_ix(results, token_ix, answer_for_token):
    results = dict(results)
    results["scores"] = results["scores"][:,:,token_ix]
    results["low_score"] = results["low_score"][token_ix]
    results["high_score"] = results["high_score"][token_ix]
    results["answer"] = answer_for_token #a bit hacky to get the answer as argument
    return results
    
def get_results_for_subject(results):
    # assumes that results already has been filtered by token_id
    results["scores"] = results["scores"][results["subject_range"][0]:results["subject_range"][1],:]
    results["input_tokens"] = results["input_tokens"][results["subject_range"][0]:results["subject_range"][1]]
    results["input_ids"] = results["input_ids"][results["subject_range"][0]:results["subject_range"][1]]
    results["subject_range"] = np.array([0,len(results["input_tokens"])])
    return results

In [18]:
token_type = "candidate"

for ix, row in queries.iloc[[45, 49]].iterrows():#queries.iterrows():
    print("=========================================================================================================")
    print(f"SAMPLE {ix}")
    if row.token_type == "any":
        
        token_ix = 0 #always measure for top predicted token (or candidate, TODO)
        answer = queries.loc[row.lama_ix]["top10_tokens"][token_ix]
    elif row.token_type == "candidate":
        print("=========================================================================================================")
        answer = queries.loc[row.lama_ix]["candidate_prediction"]
        token_ix = queries.loc[row.lama_ix]["top10_tokens"].index(answer)
        
    if row.lama_ix < results1_subj_limit:
        results_folder = results_folders[0]
    else:
        results_folder = results_folders[1]
    results_lama = np.load(os.path.join(results_folder, f"cases/{row.lama_ix}_any_mlp.npz"), allow_pickle=True)
    
    similarity_string = ["❌", "❌"]
    if row.example_more_similar == 3:
        similarity_string = ["✅", "✅"]
    elif not row.example_more_similar == 0:
        similarity_string[row.example_more_similar-1] = "✅"

    sim_scores = []
    for ix, sample_ix in enumerate([row.sample_ix_1, row.sample_ix_2]):
        proc_results_lama = get_results_for_token_ix(results_lama, token_ix, answer)
        proc_results_lama = get_results_for_subject(proc_results_lama)

        if sample_ix < results1_subj_limit:
            results_folder = results_folders[0]
        else:
            results_folder = results_folders[1]
            
        results = np.load(os.path.join(results_folder, f"cases/{sample_ix}_any_mlp.npz"), allow_pickle=True)
        results = get_results_for_token_ix(results, token_ix, answer)
        results = get_results_for_subject(results)
        # run the function below to get this, otherwise just comment out this part
        sim_scores.append(get_kl_div(proc_results_lama["scores"], results["scores"])) #get_similarity_score(proc_results_lama, results) -- assume lower is better
        print("---------------------------------------------------------------------------------------------------------")
        print(f"Example {ix+1} (top {row.token_type} token) {similarity_string[ix]} (Similarity score: {sim_scores[-1]:.4f})")
        print("---------------------------------------------------------------------------------------------------------")

        fig, axs = plt.subplots(1, 2, figsize=(8, 2), dpi=140)
        plot_trace_heatmap(proc_results_lama, axs[0])
        plot_trace_heatmap(results, axs[1])
        plt.show()
    if (not sim_scores[0]==None) and not (row.example_more_similar == 0 or row.example_more_similar == 3):
        agreement = "AGREES" if sim_scores[row.example_more_similar-1]==min(sim_scores) else "DISAGREES"
        print(f"------> The score {agreement} with the annotation.")

SAMPLE 45


AttributeError: 'Series' object has no attribute 'token_type'

In [9]:
def get_kl_div(lama_scores, scores):
    scores = get_normalized_scores(scores)
    lama_scores = get_normalized_scores(lama_scores)
    return sum(scipy.special.rel_entr(scores, lama_scores))
    
def get_normalized_scores(scores):
    delta = 0.0001
    scores = (scores.flatten()-scores.flatten().min()+delta)
    scores = scores/scores.sum()
    
    return scores